In [1]:
import os
import csv # noqa
import json
import requests
import base64
import PIL.Image as Image
from pprint import pprint
from random import sample
from urllib.request import urlretrieve
from k12libs.utils.nb_easy import k12ai_get_top_dir, RACEURL
from k12libs.utils.nb_easy import K12AI_DATASETS_ROOT

## 开发调试

In [2]:
R_PREFIX = 'https://raceai.s3.didiyunapi.com/data/datasets/cv/rmnist'
E_PREFIX = f'{K12AI_DATASETS_ROOT}/cv/rmnist/'
L_PREFIX = '/raceai/data/datasets/rmnist'
JSON_FILE = f'{R_PREFIX}/test.json'

API_INFERENCE = f'{RACEURL}/raceai/framework/inference'

In [3]:
with open(f'{E_PREFIX}/test_rotation.json', 'r') as fr:
    reader = json.load(fr)
    data_10  = sample(reader, 10) 
    data_10[0]['image_path']

In [4]:
def img2b64(path):
    with open(path, 'rb') as fr:
        b4data = base64.b64encode(fr.read()).decode()
    return b4data

In [5]:
test_pic = f"{E_PREFIX}/{data_10[0]['image_path']}"
Image.open(test_pic)

In [6]:
model = 'Resnet18'
root_dir = '/raceai/data/tmp/pl_rmnist_resnet18'
input_size = 28
mean = [
    0.1362,
    0.1362,
    0.1362
]
std = [
    0.2893,
    0.2893,
    0.2893
]
num_classes = 10
resume_weights = f"/raceai/data/ckpts/rmnist/pl_resnet18_rotation.pth"

DATA_PROCESS_BASE64 = 'raceai.data.process.Base64DataLoader'
DATA_PROCESS_PATHLIST = 'raceai.data.process.PathListDataLoader'

reqdata = '''{
    "task": "cls.inference.pl",
    "cfg": {
        "data": {
            "class_name": "%s",
            "params": {
                "data_source": %s,
                "dataset": {
                    "class_name": "raceai.data.PredictListImageDataset",
                     "params": {
                         "input_size": input_size,
                         "mean": mean,
                         "std": std
                     }
                 },
                "sample": {
                    "batch_size": 32,
                    "num_workers": 4,
                }
             }
        },
        "model": {
            "class_name": f"raceai.models.backbone.{model}",  
            "params": {
                "device": 'gpu',
                "num_classes": num_classes,
                "weights": False
            }
        },
        "trainer": {
            "default_root_dir": root_dir,
            "gpus": 1,
            "resume_from_checkpoint": resume_weights
        }
    }
}'''

### Single Image

In [7]:
for row in data_10:
    imgpath = os.path.join(L_PREFIX, row['image_path'])
    cfg = eval(reqdata % (DATA_PROCESS_PATHLIST, "[\"" + imgpath + "\"]"))
    resdata = json.loads(requests.post(url=API_INFERENCE, json=cfg).text)
    if resdata['errno'] == 0:
        result = resdata['result'][0]
        fname = result['image_path']
        probs = result['probs']
        print('{}: {} vs {}'.format(fname, row['label'], probs.index(max(probs))))
    break

/raceai/data/datasets/rmnist/imgs/test_rotation/04603.jpg: 3 vs 3


### Base64 Image

In [8]:
for row in data_10:
    imgpath = os.path.join(E_PREFIX, row['image_path'])
    b64str = img2b64(imgpath)
    cfg = eval(reqdata % (DATA_PROCESS_BASE64, f"\"{b64str}\""))
    resdata = json.loads(requests.post(url=API_INFERENCE, json=cfg).text)
    if resdata['errno'] == 0:
        result = resdata['result'][0]
        fname = result['image_path']
        probs = result['probs']
        print('{}: {} vs {}'.format(fname, row['label'], probs.index(max(probs))))
    else:
        print(resdata)

/tmp/b4img_1608128868.png: 3 vs 3
/tmp/b4img_1608128873.png: 0 vs 0
/tmp/b4img_1608128878.png: 3 vs 3
/tmp/b4img_1608128884.png: 0 vs 0
/tmp/b4img_1608128889.png: 8 vs 8
/tmp/b4img_1608128894.png: 1 vs 1
/tmp/b4img_1608128900.png: 6 vs 6
/tmp/b4img_1608128905.png: 8 vs 8
/tmp/b4img_1608128910.png: 4 vs 4
/tmp/b4img_1608128915.png: 1 vs 1


### Batch Images

In [10]:
images = [os.path.join(L_PREFIX, row['image_path']) for row in data_10]
labels = [row['label'] for row in data_10]
cfg = eval(reqdata % (DATA_PROCESS_PATHLIST, json.dumps(images, ensure_ascii=False)))
resdata = json.loads(requests.post(url=API_INFERENCE, json=cfg).text)
if resdata['errno'] == 0:
    for item, label in zip(resdata['result'], labels):
        fname = item['image_path']
        probs = item['probs']
        print('{}: {} vs {}'.format(fname, label, probs.index(max(probs))))
resdata

/raceai/data/datasets/rmnist/imgs/test_rotation/04603.jpg: 3 vs 3
/raceai/data/datasets/rmnist/imgs/test_rotation/13501.jpg: 0 vs 0
/raceai/data/datasets/rmnist/imgs/test_rotation/03536.jpg: 3 vs 3
/raceai/data/datasets/rmnist/imgs/test_rotation/26690.jpg: 0 vs 0
/raceai/data/datasets/rmnist/imgs/test_rotation/18107.jpg: 8 vs 8
/raceai/data/datasets/rmnist/imgs/test_rotation/28963.jpg: 1 vs 1
/raceai/data/datasets/rmnist/imgs/test_rotation/44172.jpg: 6 vs 6
/raceai/data/datasets/rmnist/imgs/test_rotation/51231.jpg: 8 vs 8
/raceai/data/datasets/rmnist/imgs/test_rotation/26045.jpg: 4 vs 4
/raceai/data/datasets/rmnist/imgs/test_rotation/15435.jpg: 1 vs 1


{'errno': 0,
 'result': [{'image_id': '-1',
   'image_path': '/raceai/data/datasets/rmnist/imgs/test_rotation/04603.jpg',
   'probs': [2.671597769676737e-07,
    2.2792784548641976e-08,
    1.2001068171230145e-05,
    0.9994537234306335,
    8.275234222310246e-07,
    0.00030589840025641024,
    3.5117429320052906e-07,
    7.676007385271078e-07,
    0.00021926425688434392,
    6.987701453908812e-06]},
  {'image_id': '-1',
   'image_path': '/raceai/data/datasets/rmnist/imgs/test_rotation/13501.jpg',
   'probs': [0.9999598264694214,
    1.2868697012891062e-05,
    1.5078240721777547e-05,
    1.6688156847521896e-06,
    4.6111281903904455e-08,
    2.9909119803051e-07,
    2.1967928205413045e-06,
    4.6187551561160944e-06,
    1.9073017938353587e-06,
    1.500974008195044e-06]},
  {'image_id': '-1',
   'image_path': '/raceai/data/datasets/rmnist/imgs/test_rotation/03536.jpg',
   'probs': [5.107559886141644e-08,
    1.6654720269571044e-08,
    4.1210259951185435e-06,
    0.9999351501464844

## 前端对接

步骤:

1. 选择模型(增强型,正常型)

2. 用户在画布上书写数字

3. (可选)旋转数字

4. 将最后的数字图片, 缩放到28x28大小, 并进行base64编码

5. 调用接口`/raceai/framework/inference`

## 参数配置

### 模型选择

- 旋转数字识别 (数据增强)

  cfg.trainer.resume_from_checkpoint = /raceai/data/ckpts/rmnist/pl_resnet18_rotation.pth


- 正常数字识别

  cfg.trainer.resume_from_checkpoint = /raceai/data/ckpts/rmnist/pl_resnet18_normal.pth

### Base64数据

    cfg.data.data_source = "b64str"

### 配置样例

In [11]:
reqdata = '''{
    "task": "cls.inference.pl",
    "cfg": {
        "data": {
            "class_name": "raceai.data.process.Base64DataLoader",
            "params": {
                "data_source": "%s",
                "dataset": {
                    "class_name": "raceai.data.PredictListImageDataset",
                     "params": {
                         "input_size": 28,
                         "mean": [0.1362, 0.1362, 0.1362],
                         "std": [0.2893, 0.2893, 0.2893]
                     }
                 },
                "sample": {
                    "batch_size": 32,
                    "num_workers": 4,
                }
             }
        },
        "model": {
            "class_name": f"raceai.models.backbone.Resnet18",  
            "params": {
                "device": 'gpu',
                "num_classes": 10,
                "weights": False
            }
        },
        "trainer": {
            "default_root_dir": "/raceai/data/tmp/pl_rmnist_resnet18",
            "gpus": 1,
            "resume_from_checkpoint": "/raceai/data/ckpts/rmnist/pl_resnet18_rotation.pth"
        }
    }
}''' %  img2b64(os.path.join(E_PREFIX, data_10[0]['image_path']))

print(reqdata)

{
    "task": "cls.inference.pl",
    "cfg": {
        "data": {
            "class_name": "raceai.data.process.Base64DataLoader",
            "params": {
                "data_source": "/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/wAALCAAcABwBAREA/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/9oACAEBAAA/APDNK0i/1zUIrDTbWS5uZWCqiDpkgZJ6KMkcnAGa7ez8FeGbO6sdO1jxDPc61dOIX03SrdZDBKSyhGlY7CdwUEDHJxnB3jlPFeiJ4e8T6jpUU4uI7ScxeYCDkjqMjuDkdunQdKxq1/D/AIj1Dw3cXE+myLDPPEYhOEBkiBIyUYjKnGRx1BNd54D0e1sfCkniS31fRbbXJbk29mdSuvKS3VQDJJtIKyNhl+UggZB5bC1gat4IuntrvU9P12w1sIXluRBcb7gKG+aRk5JXJyW6cE9MMeLpQCelXtK0m/1rU7fTdNtnubu4fZFEnVj16ngADJJPAAJPArvHNt8NNA1bT5njl8W6lAtrNCrF0s7dwWcE4ALn5RjJxgMCQ

## 接口请求/响应

In [15]:
resdata = json.loads(requests.post(url=API_INFERENCE, json=eval(reqdata)).text)
resdata

{'errno': 0,
 'result': [{'image_id': '-1',
   'image_path': '/tmp/b4img_1608170589.png',
   'probs': [2.6716080014921317e-07,
    2.2793003040533222e-08,
    1.2001102732028812e-05,
    0.9994537234306335,
    8.2752814023479e-07,
    0.00030589872039854527,
    3.51177988022755e-07,
    7.676058544348052e-07,
    0.00021926593035459518,
    6.98774147167569e-06]}]}